In [ ]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime, timedelta
from calendar import monthrange
%matplotlib inline


def sort_date(df):
    df['START_DATE'] = pd.to_datetime(df['START_DATE'])
    df['END_DATE'] = pd.to_datetime(df['END_DATE'])
    df.sort_values(by='START_DATE', inplace=True)


def get_all_start_end_date_ranges_in_list(df):
    num_of_rows = len(df)
    return [[df['START_DATE'].iat[i], df['END_DATE'].iat[i]] for i in range(num_of_rows)]

    
def recursive_merge(inter, start_index = 0):
    for i in range(start_index, len(inter) - 1):
        if inter[i][1] > inter[i+1][0]:
            new_start = inter[i][0]
            new_end = inter[i+1][1]
            inter[i] = [new_start, new_end]
            del inter[i+1]
            return recursive_merge(inter.copy(), start_index=i)
    return inter


def get_months_of_date_range(start, end):
    if end.day == 1:
        end = end.replace(day=2)
    return pd.date_range(start=start, end=end, freq='MS')


def cut_merged_date_ranges_by_end_of_month(merged_date_ranges):
    cutted_merged_date_ranges_by_end_of_month = []
    for start_and_end in merged_date_ranges:
        start = start_and_end[0]
        end = start_and_end[1]
        months = get_months_of_date_range(start, end)
        if not months.empty:
            for month in months:
                month = month.replace(hour=0, minute=0, second=0)
                end = month
                cutted_merged_date_ranges_by_end_of_month.append([start, end])
                start = month
            cutted_merged_date_ranges_by_end_of_month.append([month, start_and_end[1]])
        else:
            cutted_merged_date_ranges_by_end_of_month.append([start, end])
    return cutted_merged_date_ranges_by_end_of_month


def create_df_from_date_ranges(date_ranges_list):
    starts = [start_and_end[0] for start_and_end in date_ranges_list]
    ends = [start_and_end[1] for start_and_end in date_ranges_list]
    date_ranges_df = pd.DataFrame.from_dict({'START_DATE': starts, 'END_DATE': ends})
    return date_ranges_df


def calculate_month_percentage(row):
    first_day_pf_month = row[0]
    next_month = first_day_pf_month.replace(day=28) + dt.timedelta(days=4)
    last_day = next_month - dt.timedelta(days=next_month.day)
    days_of_month = last_day.day
    total_time = row[1]
    total_time_in_sec = int(total_time.total_seconds())
    month_in_sec = days_of_month*86400
    return (total_time_in_sec/month_in_sec)*100


    
    
df = pd.read_csv('leaves.csv')
sort_date(df)
start_end_ranges = get_all_start_end_date_ranges_in_list(df)
merged_start_end_list = recursive_merge(start_end_ranges)
cutted_merged_start_end_list = cut_merged_date_ranges_by_end_of_month(merged_start_end_list)
df = create_df_from_date_ranges(cutted_merged_start_end_list)
sort_date(df)
df['DIFFERENCE'] = (df['END_DATE'] - df['START_DATE'])
df = df.groupby(df['START_DATE'].dt.strftime('%Y-%m')).agg(TOTAL_TIME = ('DIFFERENCE', 'sum'))
df.reset_index(inplace=True)
df['START_DATE'] = pd.to_datetime(df['START_DATE'])
df['PERCENTAGE'] = df.apply(calculate_month_percentage, axis=1)
df['START_DATE'] = df['START_DATE'].dt.strftime('%Y-%m')
print(df)



